In [1]:
import tensorflow as tf
import numpy as np

s = tf.InteractiveSession()

In [2]:
from model.kb_population import *

In [3]:
model = sample_model(s)

In [4]:
map(lambda v: v.name, tf.trainable_variables())

[u'fact/left/RNN/EmbeddingWrapper/embedding:0',
 u'fact/left/RNN/MultiRNNCell/Cell0/BasicLSTMCell/Linear/Matrix:0',
 u'fact/left/RNN/MultiRNNCell/Cell0/BasicLSTMCell/Linear/Bias:0',
 u'fact/right/RNN/EmbeddingWrapper/embedding:0',
 u'fact/right/RNN/MultiRNNCell/Cell0/BasicLSTMCell/Linear/Matrix:0',
 u'fact/right/RNN/MultiRNNCell/Cell0/BasicLSTMCell/Linear/Bias:0',
 u'query/left/RNN/EmbeddingWrapper/embedding:0',
 u'query/left/RNN/MultiRNNCell/Cell0/BasicLSTMCell/Linear/Matrix:0',
 u'query/left/RNN/MultiRNNCell/Cell0/BasicLSTMCell/Linear/Bias:0',
 u'query/right/RNN/EmbeddingWrapper/embedding:0',
 u'query/right/RNN/MultiRNNCell/Cell0/BasicLSTMCell/Linear/Matrix:0',
 u'query/right/RNN/MultiRNNCell/Cell0/BasicLSTMCell/Linear/Bias:0',
 u'classification/attention/w_proj:0',
 u'classification/attention/b_proj:0',
 u'classification/attention/w_att:0',
 u'classification/attention/w_score:0',
 u'classification/w_q_inter:0',
 u'classification/w_a_inter:0',
 u'classification/b_inter:0',
 u'classif

In [5]:
model.fact_left_rnns[1].grads

[[<tensorflow.python.framework.ops.IndexedSlices at 0x7f2dd6690b50>,

In [6]:
s.run(tf.initialize_all_variables())

In [13]:
s.run(tf.trainable_variables()[4])

array([[ 0.02800972,  0.09775228, -0.01115346, -0.00649111, -0.05368328,
         0.01207046,  0.07981195, -0.03507855],
       [ 0.08302356,  0.04159   , -0.05133739, -0.01720567,  0.08242323,
         0.05165475,  0.04352088, -0.08779841],
       [ 0.00868559,  0.05444861, -0.05486369, -0.09518442,  0.09478817,
         0.03224761,  0.06359459, -0.05946524],
       [ 0.07140889,  0.01672158,  0.03913409, -0.07339261,  0.08426569,
         0.0890832 ,  0.04402249, -0.02796014]], dtype=float32)

In [1]:
import tensorflow as tf
import numpy as np
from model.kb_population import *

s = tf.InteractiveSession()

batch_size = 2
model = sample_model(s, batch_size=batch_size, forward_only=False)
s.run(tf.initialize_all_variables())

 # ----- FORWARD ------
# facts
left_fact_bucket_ids = [0, 0, 0]
right_fact_bucket_ids = [0, 0, 0]

fact_inputs = {}
fact_outputs = []
for i in xrange(model.max_facts):
    fact_outputs.append(model.fact_left_rnns[i].outputs[left_fact_bucket_ids[i]])
    fact_outputs.append(model.fact_right_rnns[i].outputs[right_fact_bucket_ids[i]])
    fact_inputs[model.fact_left_rnns[i].inputs[0]] = np.zeros([batch_size])  # Example
    fact_inputs[model.fact_right_rnns[i].inputs[0]] = np.ones([batch_size])  # Example

dep_fact_tensors = get_dep_tensors(fact_outputs, fact_inputs.keys())
dep_fact_tensors = fact_outputs + [t for t in dep_fact_tensors if t not in fact_outputs]
fact_state = s.run(dep_fact_tensors, fact_inputs)

# concatenate left and right fact embeddings for each fact
concat_facts = []
for i in xrange(model.max_facts):
    concat_facts.append(np.concatenate((fact_state[2*i], fact_state[2*i+1]), 1))

In [2]:
# Query
left_query_bucket_id = 0
right_query_bucket_id = 0

query_inputs = {}
query_outputs = [model.query_left_rnn.outputs[left_query_bucket_id],
                 model.query_right_rnn.outputs[right_query_bucket_id]]

query_inputs[model.query_left_rnn.inputs[0]] = np.zeros([batch_size])  # Example
query_inputs[model.query_right_rnn.inputs[0]] = np.ones([batch_size])  # Example

dep_query_tensors = list(get_dep_tensors(query_outputs, query_inputs.keys()))
dep_query_tensors = query_outputs + filter(lambda t: t not in query_outputs, dep_query_tensors)
query_state = s.run(dep_query_tensors, query_inputs)

concat_query = np.concatenate((query_state[0], query_state[1]), 1)

In [3]:
# Classification
classification_input = dict(zip(model.attention_states_input, concat_facts))
classification_input[model.query_input] = concat_query
classification_input[model.neg_candidates] = np.array([[1], [0]])
classification_input[model.labels] = np.array([0,1])
classification_input[model.sampling_prob] = np.array([[1.0], [1.0]])

grads = s.run([model.update, model.loss, model.query_input_grad] + model.attention_states_grads,
              feed_dict=classification_input)

loss = grads[1]
query_grad = grads[2]
fact_grads = grads[3:]

In [4]:
# facts
fact_updates = []
fact_inputs_u = dict(fact_inputs)
for i in xrange(model.max_facts):
    g = fact_grads[i]
    fact_inputs_u[model.fact_left_rnns[i].out_grad] = g[:, :g.shape[1]/2]
    fact_inputs_u[model.fact_right_rnns[i].out_grad] = g[:, g.shape[1]/2:]
    fact_updates.append(model.fact_left_rnns[i].updates[left_fact_bucket_ids[i]])
    fact_updates.append(model.fact_right_rnns[i].updates[right_fact_bucket_ids[i]])

for i in xrange(len(dep_fact_tensors)):
    fact_inputs_u[dep_fact_tensors[i]] = fact_state[i]

s.run(fact_updates, fact_inputs_u)

[None, None, None, None, None, None]

In [5]:
 # query
query_updates = []
query_updates.append(model.query_left_rnn.updates[left_query_bucket_id])
query_updates.append(model.query_right_rnn.updates[right_query_bucket_id])
#query_updates.append(model.query_left_rnn.grads[left_query_bucket_id][1])
#query_updates.append(model.query_right_rnn.grads[left_query_bucket_id][1])
query_inputs_u = dict(query_inputs)
query_inputs_u[model.query_left_rnn.out_grad] = query_grad[:, :query_grad.shape[1]/2]
query_inputs_u[model.query_right_rnn.out_grad] = query_grad[:, query_grad.shape[1]/2:]

for i in xrange(len(dep_query_tensors)):
    query_inputs_u[dep_query_tensors[i]] = query_state[i]

s.run(query_updates, query_inputs_u)

[None, None]

In [6]:
map(lambda t: (t[0].name,t[1]) , zip(dep_query_tensors, query_state))

[(u'query/left/RNN/MultiRNNCell/Cell0/BasicLSTMCell/mul_2:0',
  array([[-0.00039665,  0.00037709],
         [-0.00039665,  0.00037709]], dtype=float32)),
 (u'query/right/RNN/MultiRNNCell/Cell0/BasicLSTMCell/mul_2:0',
  array([[ 0.00070209,  0.00196058],
         [ 0.00070209,  0.00196058]], dtype=float32)),
 (u'query/right/RNN/pack/values_1:0', 4),
 (u'query/left/RNN/MultiRNNCell/Cell0/BasicLSTMCell/Sigmoid_2:0',
  array([[ 0.50017822,  0.49971548],
         [ 0.50017822,  0.49971548]], dtype=float32)),
 (u'query/right/RNN/Slice:0', array([2], dtype=int32)),
 (u'query/right/RNN/MultiRNNCell/Cell0/BasicLSTMCell/Linear/Bias/read:0',
  array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.], dtype=float32)),
 (u'query/right/RNN/MultiRNNCell/Cell0/Slice/size:0',
  array([-1,  4], dtype=int32)),
 (u'query/right/RNN/MultiRNNCell/Cell0/BasicLSTMCell/mul:0', array([[ 0.,  0.],
         [ 0.,  0.]], dtype=float32)),
 (u'query/right/RNN/MultiRNNCell/Cell0/BasicLSTMCell/split:0',
  array([[ 0.,  0.],
    

In [10]:
def get_dep_tensors(output_tensors, input_tensors, current=None):
    res = set()
    for o in output_tensors:
        if o not in input_tensors:  # we do not want to add placeholders, for example
            current_new = set()
            current_new.add(o)
            if current:
                current_new = current_new.union(current)
            res = res.union(get_dep_tensors(o.op.inputs, input_tensors, current_new))
        else:
            res = res.union(current)
    return res

In [12]:
len(map(lambda t: t.name, get_dep_tensors(query_outputs, query_inputs.keys())))

54